In [1]:
import os
import mlflow
from mlflow import log_metric, log_param

In [2]:
mlrun_dir = '../../cluster_runs (copy)/.'

In [3]:
os.chdir(mlrun_dir)

In [4]:
exp_names = ('it_tunning', 'fr_tunning', 'ru_tunning', 'ge_tunning', 'en_tunning_search_iters_100')
model_list= ['LogisticRegression', 'LogisticRegressionRidge', 'LogisticRegressionRidgeDual', 'LogisticRegressionLasso', 'LogisticRegressionElasticNet', 'RidgeClassifier', 'SVM', 'LinearSVM', 'LinearSVMDual', 'RandomForest', 'XGBoost', 'RandomForestV2', 'ComplementNaiveBayes', 'NaiveBayes']
model_list= ['LogisticRegression', 'LogisticRegressionRidgeDual', 'LogisticRegressionLasso', 'LogisticRegressionElasticNet', 'RidgeClassifier', 'SVM', 'LinearSVM', 'LinearSVMDual', 'RandomForest', 'XGBoost', 'RandomForestV2', 'ComplementNaiveBayes', 'NaiveBayes']

## Separate single runs into multiple experiments

### Get dictionary of experiment names to their codes

In [5]:
from mlflow import MlflowClient
from mlflow.entities import ViewType

In [6]:
client =  MlflowClient()

In [ ]:
analysis_level = 'inner_cv'
query = "params.analysis_level = '{analysis_level}' and params.model_name = '{model_name}'"
for exp_name in exp_names:
    exp_id = client.search_experiments(filter_string=f"name = '{exp_name}'")[0].experiment_id
    for model_name in model_list:
        print(f"{exp_name}_{model_name}_{analysis_level}")
        try:
            new_exp_id = client.create_experiment(f"{exp_name}_{model_name}_{analysis_level}")

            runs = client.search_runs(
            experiment_ids=[exp_id],
            filter_string=query.format(model_name=model_name, analysis_level=analysis_level),
            run_view_type=ViewType.ACTIVE_ONLY)

            # Store each of the runs
            for run in runs:
                with mlflow.start_run(experiment_id=new_exp_id):
                    for metric_name, metric_value in run.data.metrics.items():
                        log_metric(metric_name, metric_value)

                    for param_name, param_value in run.data.params.items():
                        log_param(param_name, param_value)

                mlflow.end_run()
        except Exception as e:
            print(e)
            pass

it_tunning_LogisticRegression_inner_cv


In [24]:
exp = client.search_experiments(filter_string="name = 'it_tunning'").experiment_id

In [26]:
exp[0].artifact_location

'/net/cephfs/data/jberme/policy-framing/mlruns/7'

In [27]:
exp[0].experiment_id

'7'

In [28]:
query = "params.analysis_level = 'outer_cv' and params.model_name = 'LogisticRegression'"
runs = client.search_runs(experiment_ids=["7"], filter_string=query, run_view_type=ViewType.ACTIVE_ONLY)

In [30]:
type(runs)

mlflow.store.entities.paged_list.PagedList

In [44]:
for run in runs:
    print(run.data.metrics.items())
    break

dict_items([('train_recall_macro', 1.0), ('train_accuracy', 1.0), ('train_f1_macro', 1.0), ('test_f1_macro', 0.3630890603603488), ('test_precision_macro', 0.39776440490726195), ('train_precision_micro', 1.0), ('train_precision_macro', 1.0), ('test_precision_micro', 0.46681929181929177), ('test_accuracy', 0.05063291139240506), ('test_recall_macro', 0.3604607876159601), ('train_recall_micro', 1.0), ('train_f1_micro', 1.0), ('test_recall_micro', 0.458041958041958), ('test_f1_micro', 0.4531988325418664)])


In [45]:
runs[0].data.params

{'min_df': '0.05',
 'multilabel_type': 'independent',
 'base_estimator_name': "LogisticRegression(penalty='none')",
 'model_name': 'LogisticRegression',
 'min_var': '0.001',
 'n_features': '295',
 'tfidf': 'True',
 'class_weight': 'balanced',
 'language': 'it',
 'corr_threshold': '0.9',
 'analysis_level': 'outer_cv',
 'spacy_model_used': 'it_core_news_sm',
 'max_df': '0.95',
 'vectorizer_max_features': '10000',
 'max_iter': '29700',
 'is_count_vectorizer': 'False',
 'unit_of_analysis': 'title_and_10_sentences',
 'n_gram_range_start': '1',
 'n_gram_range_end': '1'}